### 국평오 이슈
- 우선 문제를 이해한게 맞는지 확인해보고 싶었다.
  - 각 자릿수의 차이가 정확히 1 이라고 한다. 그렇다면 101001 은 계단수가 아니고, 101010만 계단수가 될 것이다.
  - 그러면 $B=2$ 일 때 $N$ 에 상관없이 답이 1일 것이다. 
    - 이전 자릿수가 0이였다면, 다음 자릿수는 반드시 1이여야 하기 때문. vice-versa
    - 그리고 그렇게 배치하면 반드시 1번은 가능할거고, 또한 그것이 유일한 방법일 것이다.
- 예제를 보니 0인 경우도 있는듯하다. 어떨 때 그럴까?
  - 에잉? 역시 문제를 잘못 읽은게 맞다.


### 아이디어 
- `모든 한 자리 숫자가 자릿수로 등장하면서` 라는 조건이 붙어있다. 조금 표현이 이상하지만, 이해는 되는 것 같다.
- $B=2$ 였다면, 0과 1이 모두 등장해야 한다.
  - 그러면 $N=1$ 일 땐 불가능하게 될 것이다. 적어도 2자리 이상이어야 한다.
  - 이를 확장하면, $B > N$ 일 때 답이 0이다.
- $B=N$ 일 때는 답이 1이다. 
  - $B-1$ 부터 0까지 내림차순으로 정렬된 숫자들 하나만 된다.
  - 위 방법은 오름차순으로도 할 수 있지만, 0으로 시작하는 숫자가 없다고 했으므로 1이다.
- $B=N+1$ 일 때에는 답이 3이다.
  - `왔다가 돌아가는` 패턴으로 배치할 수 있다. 
    - ex) `7 6` 일 때, `4543210`, `1012345`, `5432101` 이 가능하다.
    - 이 패턴의 방법 외로 계단수를 만드는 방법이 없다.
    - 이전 방법에서 양 끝에 있는 수에서 1만큼 차이나는 숫자를 붙였다는 점에 주목해야 할 것 같다.
  - 그런데 $B=2$ 일 때는 답이 `101`로 1이다. 이것만 예외인 이유는 0으로 시작할 수 없기 때문.
    - 좀 더 생각해보니 $B = 2$ 라면 trivial하게 1이다. 1010...으로 반복되는 패턴 외에 계단수가 아니다. 이것만 예외로 두자.
- $B=N+2$ 는 몇일까? 
  - 예제 1에서 그 답이 3 + (N-1) 인것을 역이용해보자.
    - 더 해보기 전에 내 손을 못믿으니, 브루트포스 코드로 패턴을 보는게 좋을수도 있다.
    - $B=3$ 일 때, `3, 5, 9, 13, 21`
      - delta가 2씩 증가한다. delta의 delta는 0이다.
    - $B=4$ 일 때, `3, 8, 17, 35` 
  - 수치해석은 답이 안나온다. 그냥 답지를 봐야겟다.

5 4
;;
6 4
;;
7 4
;;
8 4
;;
9 4
;;
10 4
;;
11 4
;;
12 4

In [ ]:
import io, os
input=io.BytesIO(os.read(0,os.fstat(0).st_size)).readline

MOD = int(1e9)
NONE = -1
def sol() :
  N, B = map(int, input().split())
  DP = [[[[NONE] * 2 for _ in range(2)] for _ in range(N+7)] for _ in range(N+7)]
  
  def f(i, v, z, b) :
    __ = (i, v, z, b) #초기값 저장
    S = [(i, v, z, b)]
    while S :
      i, v, z, b = S.pop()

      if i == N : #최종적으로 모든 수가 다 나왔을 때 현재 상태가 가능한 것으로 표시한다
        DP[i][v][z][b] = z & b
        continue
      
      DP[i][v][z][b] = 0 #기저상태. 가능한지의 여부
      if v > 0 : #현재 수보다 작은 수를 뽑을 수 있다면
        dm = DP[i+1][v-1][z | (v == 1)][b] #delta minus
        if dm == NONE :
          S.append((i, v, z, b))
          S.append((i+1, v-1, z | (v == 1), b)) #만약 그 수가 0이라면, 그 수가 나왔다는 것을 표시한다
          continue
        
        DP[i][v][z][b] += dm
        DP[i][v][z][b] %= MOD
      
      if v < B-1 : #현재 수보다 큰 수를 뽑을 수 있다면
        dp = DP[i+1][v+1][z][b | (v == B-2)]
        if dp == NONE:
          S.append((i, v, z, b))
          S.append((i+1, v+1, z, b | (v == B-2))) #만약 그 수가 N-1이라면, 그 수가 나왔다는 것을 표시한다
          continue
        
        DP[i][v][z][b] += dp
        DP[i][v][z][b] %= MOD

    i, v, z, b = __ 
    return DP[i][v][z][b]

  ans = 0
  for i in range(1, B) :
    ans += f(1, i, 0, i == B-1)
    ans %= MOD
  
  print(ans)
    

sol()

In [ ]:
import io, os
input=io.BytesIO(os.read(0,os.fstat(0).st_size)).readline

MOD = int(1e9)
NONE = -1
def sol() :
  N, B = map(int, input().split())
  DP = [[[0 for _ in range(2)] for _ in range(2)] for _ in range(B)]
  for v in range(1, B):
    DP[v][0][v == B-1] = 1 #b-1으로부터 시작이면, 그 flag를 미리 1로 켜야한다.

  for _ in range(1, N):
    nDP = [[[0 for _ in range(2)] for _ in range(2)] for _ in range(B)]
    for v in range(B):
      for z in range(2):
        for b in range(2):
          cnt = DP[v][z][b]
          if cnt > 0:
            if v > 0:
              nDP[v-1][z or v-1 == 0][b or v-1 == B-1] += cnt
              nDP[v-1][z or v-1 == 0][b or v-1 == B-1] %= MOD
            if v < B-1:
              nDP[v+1][z or v+1 == 0][b or v+1 == B-1] += cnt
              nDP[v+1][z or v+1 == 0][b or v+1 == B-1] %= MOD
    DP = nDP

  ans = sum(DP[v][1][1] for v in range(B)) % MOD
    
  print(ans)

sol()

- 3898. 2번째 예제에서 S의 최대 길이였다. 엄청난 메모리를 먹는 것이 원인인가? 재귀로 풀 수가 없다.
- for 비재귀로 바꿔야겠다.. DP 비트필드 문제의 형식을 좀 따르면 될 것이다.

### 풀이
- 시간복잡도 $O(NB)$
  - 즉, 각 자리의 $N$ 개의 숫자를 $B$ 번 넣어봐야 알 수 있단 뜻이다.
- `DP[i][v][z][b]` = `i`번째 자리수까지 채웠을 때, `v`가 마지막으로 등장하고, $0$ 이 등장했는지 여부 `z`, 진법의 마지막 수 ($B-1$) 가 등장했는지 여부 `b`
  - `v` 가 0 이상이면 다음 자릿수의 숫자를 줄이는 방향으로 확장할 수 있고, `B-1` 이하면 늘리는 방향으로 확장할 수 있다.
  - 계단수가 가능한지의 여부는 `z`와 `b`가 둘다 켜져있는지만 보면 된다. 
    - (증명) 문제의 정의에 의해 매번 인접한 숫자, 즉 `i+1` 에서 $v \pm 1$ 으로만 바뀔 수 있다.
    - 인접한 숫자들만 고르므로, 그 사이의 숫자들을 모두 방문할 수 밖에 없다.
    - 그렇지 않고 `z` 와 `b`에 동시에 도달할 수 없다. 따라서 성립.
  - `z, b` 가 둘다 켜져있으려면 최소한 `z`나 `b` 에서 시작해서 `b` 번의 step 이후 반대쪽에 도달할 수 있다.

### 최적화
- 전이할 때 바로 이전 상태의 값만 보면 된다. 
  - 따라서 한차원 줄일 수 있다.
  - 단, 토글링은 안되고, 매번 새로운 배열을 선언해야 한다.
- 조건이 여러개 달려있는 DP 타입의 문제이다. `z, b` 를 하나의 비트셋으로 합칠 수 있다.
  - $z, b \in GF(2)$
    - $z, b \in \{0, 1\}$ 이므로, $z, b \in GF(2)$ 이다.
    - 그니까 $z, b$ 는 `boolean` 타입이라는 뜻.
  - $k \in GF(2^2)$ consist of $(z, b)$
    - $k$ 를 튜플(순서쌍) $(z, b)$ 로 이루어진 이진 가환체 $GF(2^2)$ 로 정의한다
- $DP[i][k]$: $i$ 번째 자릿수에서 상태가 $k$ 일 때 만들 수 있는 계단수의 개수

In [ ]:
import io, os
input=io.BytesIO(os.read(0,os.fstat(0).st_size)).readline

MOD = int(1e9)
NONE = -1
def sol() :
  N, B = map(int, input().split())
  DP = [[0] * 4 for _ in range(B)]
  for v in range(1, B):
    DP[v][v == B-1] = 1

  for _ in range(1, N):
    nDP = [[0] * 4  for _ in range(B)]
    for v in range(B):
      for k in range(4):
        cnt = DP[v][k]
        if cnt > 0:
          if v > 0: 
            nk = k | (v-1 == 0) << 1 | v-1 == B-1
            nDP[v-1][nk] += cnt
            nDP[v-1][nk] %= MOD
          if v < B-1:
            nk = k | (v+1 == 0) << 1 | v+1 == B-1
            nDP[v+1][nk] += cnt
            nDP[v+1][nk] %= MOD
    DP = nDP

  ans = sum(DP[v][3] for v in range(B)) % MOD
    
  print(ans)

sol()

- 접근 자체를 잘못한게 큰 문제였다. 이쪽 패턴을 이제야 정리할 수 있었다.
  - 조건 달린 DP 완탐?